## Bring in Alignment for mapping

This program will map TFBS using the Biopython's motif package.

**Inputs**: 
1. before alignment (fasta) 
2. after alignment (fasta) 
3. TFBS Position Frequency Matrix.

**Outputs**:
1. `.csv` file that outputs found TFBSs at each position, if any, in alignment (position, score, species, raw_position, strand motif_found)
2. `.csv` file that outputs only TFBS found.

**To Do**:
- [ ] Make Vector that attaches
- [ ] Attach real species name
- [ ] Loop through all files in directory
- [ ] Write file name in last column
- [ ] Append to a file 
- [ ] Have all of the important checks read out into a file

In [21]:
from Bio import motifs
from Bio import SeqIO 
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC, generic_dna, generic_protein
from collections import defaultdict
import re
import pandas as pd
import numpy as np
import os, sys

In [22]:
####################
## Functions
#####################

## Print full DF
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [ ]:
#####################
## sys Inputs - to do
#####################

## So I can run in shell and loop through sequences AND motifs to get a giant dataset

### input1 = open(sys.argv[1]) #file that contains text
## input2 = open(sys.argv[2]) #file that contains terms to search for

In [23]:
##################
## Input 1 - Alignment Input 
##################

## read in alignment as a list of sequences
alignment = list(SeqIO.parse("../data/fasta/output_ludwig_eve-striped-2.fa", "fasta"))

## Get alignment ID list (for later)
alignment_id = []
for seq in alignment:
    alignment_id.append(str(seq.id))
    
## Check 
print("Found %i records in alignment file" % len(alignment))

## Sequence Length should be the same for all alignment sequences
for seq in alignment:
    print len(seq)
    
## Turn sequences into a list of strings
## Note: They are no longer bio.seq.seq objects
alignment_string_list = []
for seq in alignment:
    alignment_string_list.append(str(seq.seq))

Found 9 records in alignment file
1136
1136
1136
1136
1136
1136
1136
1136
1136


In [24]:
#####################
## Input 2 - Raw Sequences Input
#####################

## This is used to search for TFBSs in sequences

raw_sequences = list(SeqIO.parse("../data/fasta/ludwig_eve-striped-2.fasta", "fasta"))
print("Found %i records in raw sequence file" % len(raw_sequences))

## make all IUPAC.IUPACUnambiguousDNA()
raw_sequences_2 = []
for seq in raw_sequences:
    raw_sequences_2.append(Seq(str(seq.seq), IUPAC.IUPACUnambiguousDNA()))
    print len(seq)

Found 9 records in raw sequence file
928
875
898
868
862
913
905
909
868


In [25]:
#####################
## Input 3 - Motif Input
#####################

motif = motifs.read(open("../data/PWM/transpose_fm/bcd_FlyReg.fm"),"pfm")
motif.weblogo("mymotif.png")
print(motif.counts)
pwm = motif.counts.normalize(pseudocounts=0.0) # Doesn't change from pwm
pssm = pwm.log_odds()
print(pssm) # Why do I need log odds exactly?
motif_length = len(motif) #for later retrival of nucleotide sequence


        0      1      2      3      4      5      6      7
A:   0.19   0.17   0.88   0.92   0.04   0.04   0.06   0.12
C:   0.37   0.08   0.04   0.02   0.02   0.87   0.52   0.25
G:   0.08   0.04   0.04   0.04   0.33   0.02   0.08   0.37
T:   0.37   0.71   0.04   0.02   0.62   0.08   0.35   0.27

        0      1      2      3      4      5      6      7
A:  -0.38  -0.53   1.82   1.88  -2.70  -2.70  -2.12  -1.12
C:   0.55  -1.70  -2.70  -3.70  -3.70   1.79   1.05  -0.00
G:  -1.70  -2.70  -2.70  -2.70   0.39  -3.70  -1.70   0.55
T:   0.55   1.51  -2.70  -3.70   1.30  -1.70   0.47   0.11



In [26]:
######################
## Searching for Motifs in Sequences
######################

## Returns a list of arrays with a score for each position

## This give the score for each position
## If you print the length you get the length of the sequence minus TFBS length. 

## Forward stand

pssm_list = [ ]
for seq in raw_sequences_2:
    pssm_list.append(pssm.calculate(seq))

## Check
## You should notice that the length is total length of raw_sequence minus length of motif.
## for seq in pssm_list:
##    print len(seq)
    
## Reverse strand
## rpssm = pssm.reverse_complement()

In [27]:
########################## 
## Automatic Calculation of threshold
##########################

## Ideal to find something that automatically calculates, as
## opposed to having human choosing.

## Approximate calculation of appropriate thresholds for motif finding 
## Patser Threshold
## It selects such a threshold that the log(fpr)=-ic(M) 
## note: the actual patser software uses natural logarithms instead of log_2, so the numbers 
## are not directly comparable. 

distribution = pssm.distribution(background=motif.background, precision=10**4)
patser_threshold = distribution.threshold_patser() #for use later

print("Patser Threshold is %5.3f" % patser_threshold) # Calculates Paster threshold. 

Patser Threshold is 3.262


In [28]:
###################################
## Searching for motif in all raw_sequences
#################################

position_list = []
for i in range(0,8):
    for position, score in pssm.search(raw_sequences_2[i], threshold = patser_threshold):
        positions = {'score':score, 'position':position, 'species': i}
        position_list.append(positions)
        
position_DF = pd.DataFrame(position_list)

## Check
## print position_DF

In [29]:
#############################
## Add strand and pos position information as columns to position_DF
##############################

## raw_position column
## change position to positive
position_list_pos = []
for x in position_DF['position']:
    if x < 0:
       position_list_pos.append(905 + x)
    else:
       position_list_pos.append(x)

## append to position_DF
position_DF['raw_position'] = position_list_pos
    
## strand Column
strand = []
for x in position_DF['position']:
    if x < 0:
       strand.append("negative")
    else:
       strand.append("positive")
    
## append to position_DF
position_DF['strand'] = strand

## motif_found column
## First turn into a list of strings
raw_sequences_2_list = []
for seq in raw_sequences_2:
    raw_sequences_2_list.append(str(seq))

## Now get all motifs found in sequences
motif_found = []
for x in position_DF['position']:
    motif_found.append(raw_sequences_2_list[i][x:x + motif_length])

## append to position_DF
position_DF['motif_found'] = motif_found


## Check
## len(motif_found)    
## print(motif_found) 
## print(position_DF)

In [30]:
##################
## get alignment position 
#################

## Need to map to the sequence alignment position

remap_list = []
nuc_list = ['A', 'a', 'G', 'g', 'C', 'c', 'T', 't', 'N', 'n']


positions = {'score':score, 'position':position, 'species': i}
position_list.append(positions)


for i in range(0,8):
    counter = 0
    for xInd, x in enumerate(alignment_string_list[i]):    
        if x in nuc_list:
            remaps = {'raw_position': counter, 'align_position':xInd, 'species':i}
            counter += 1
            remap_list.append(remaps)
remap_DF = pd.DataFrame(remap_list)
            
## Check
## print_full(remap_DF)

In [31]:
## Merge both datasets

## Check first
print(position_DF.shape)
print(remap_DF.shape)

## Merge - all sites
TFBS_map_DF_all = pd.merge(position_DF, remap_DF, on=['species', 'raw_position'], how='outer')
TFBS_map_DF_all = TFBS_map_DF_all.sort_values(by=['species','align_position'], ascending=[True, True])

## Check
## print_full(TFBS_map_DF_all)
## print(TFBS_map_DF_all.shape)

# Merge - only signal 
TFBS_map_DF_only_signal = pd.merge(position_DF, remap_DF, on=['species', 'raw_position'], how='inner')
TFBS_map_DF_only_signal = TFBS_map_DF_only_signal.sort_values(by=['species','align_position'], ascending=[True, True])

## To quickly check if species share similar TFBS positions
print(TFBS_map_DF_only_signal.sort_values(by=['raw_position'], ascending=[True]))


## Check
## print_full(TFBS_map_DF_only_signal)
## print(TFBS_map_DF_only_signal.shape)

(234, 6)
(7098, 3)
     position      score  species  raw_position    strand motif_found  \
0          10   5.013668        0            10  positive    ataatttt   
85         10   5.454241        3            10  positive    ataatttt   
111        10   5.013668        4            10  positive    ataatttt   
30         10   5.013668        1            10  positive    ataatttt   
139        10   5.454241        5            10  positive    ataatttt   
166        10   5.013668        6            10  positive    ataatttt   
200        10   5.013668        7            10  positive    ataatttt   
57         10   5.013668        2            10  positive    ataatttt   
58         76   4.559102        2            76  positive    ggcactcc   
201        89   4.894493        7            89  positive    cgaagcct   
167      -804   3.414712        6           101  negative    ggaactga   
86        144  10.457056        3           144  positive    acatgaag   
168       145  10.457056        

In [ ]:
####################
## Attach input files name as a column
#####################

## Ideally I would attach the file name of the 1. raw sequence and 2. the motif being tested


